In [1]:
from google.colab import files
files.upload()  # subís kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tatianacandia","key":"a1b3258c73ca68c1799cb5305927f55f"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d msambare/fer2013
!unzip -q fer2013.zip -d fer2013


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
  0% 0.00/60.3M [00:00<?, ?B/s]
100% 60.3M/60.3M [00:00<00:00, 1.76GB/s]


Crear dataloaders de FER2013

In [4]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Transformaciones FER2013 → EfficientNet (224x224, RGB)
transform = transforms.Compose([
    transforms.Grayscale(3),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# RUTAS (ajusta si están en otro lado)
train_path = "/content/fer2013/train"
val_path   = "/content/fer2013/test" # Changed 'val' to 'test'

train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset   = datasets.ImageFolder(val_path,   transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Cargado FER2013 ✔")
print("Train images:", len(train_dataset))
print("Val images:", len(val_dataset))


Cargado FER2013 ✔
Train images: 28709
Val images: 7178


In [6]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0

class MicroExpNet(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.model = efficientnet_b0(weights=None)
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Crear el modelo
model = MicroExpNet()

# Cargar checkpoint
state = torch.load("/content/best_microexp_model.pth", map_location="cpu")

# Crear nuevo diccionario con prefijo "model."
new_state = {}
for k, v in state.items():
    new_key = "model." + k  # <-- AGREGAMOS EL PREFIJO
    new_state[new_key] = v

# Cargar correctamente
model.load_state_dict(new_state, strict=True)

print("✔ Modelo cargado correctamente con prefijo añadido")


✔ Modelo cargado correctamente con prefijo añadido


# Entrenamiento

In [8]:
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn

# --- FIX: Define missing variables ---
num_epochs = 10  # You can adjust this value
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001) # Example optimizer
criterion = nn.CrossEntropyLoss() # Example loss function
# -------------------------------------

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({"loss": loss.item()})

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}")

Epoch 1/10:   0%|          | 0/898 [00:00<?, ?it/s]

Epoch [1/10] - Loss: 1.1277


Epoch 2/10:   0%|          | 0/898 [00:00<?, ?it/s]

Epoch [2/10] - Loss: 0.9403


Epoch 3/10:   0%|          | 0/898 [00:00<?, ?it/s]

...

Epoch [8/10] - Loss: 0.4350


Epoch 9/10:   0%|          | 0/898 [00:00<?, ?it/s]

Epoch [9/10] - Loss: 0.3653


Epoch 10/10:   0%|          | 0/898 [00:00<?, ?it/s]

Epoch [10/10] - Loss: 0.3066


Validacion

In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy de validación: {accuracy:.2f}%")


  0%|          | 0/225 [00:00<?, ?it/s]

Accuracy de validación: 66.59%


In [10]:
torch.save(model.state_dict(), "/content/microexp_retrained_FER2013.pth")
print("Modelo guardado ✔")

Modelo guardado ✔
